# Getting available actions
Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append("..")
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import sys, os


%load_ext lab_black

In [2]:
import app.ajaxviews.actions as actions
import app.models as models

c = models.CosmosdbClient()

You can load the full actions list. 

In [3]:
all_actions = actions.get_actions_config()
pd.DataFrame(all_actions)

,type,label,applies_to,effort,requires_attr,augments_self_properties,comment,leadingdesire
0,patriot_education,action,pop,10,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.01, 'ag...",expand public education programs with a collec...,NaN
1,safety_nets,action,pop,15,{'wealth': 0.1},"{'constitution': 0.05, 'wealth': '-.05'}",increase the general health of the population,NaN
2,individual_education,action,pop,10,{'wealth': 0.1},"{'conformity': -0.05, 'literacy': 0.1}",increase the general health of the population,NaN
3,patriot_propoganda,action,pop,10,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt...",NaN
4,build_infrastructure,action,pop,20,{'wealth': 0.5},"{'wealth': -0.05, 'industrial_infrastructure': 1}",increase industry by investing in local infras...,wealth
5,healthcare_initialtives,action,pop,15,{'wealth': 0.1},"{'health': 0.01, 'wealth': '-.01'}",increase the health and literacy of other popu...,NaN
6,individual_education,action,pop,15,{'wealth': 0.1},"{'wealth': '-.1', 'literacy': 0.01, 'factionLo...","expand privatized education, increase literacy...",NaN


In [4]:
pop_id = "8880560711424"
c.run_query(f"g.V().has('objid','{pop_id}').valueMap()")
pop = c.clean_nodes(c.res)[0]
pop

{'isIdle': 'false',
 'name': 'Isgi Argorflo',
 'objid': '8880560711424',
 'conformity': 0.489,
 'literacy': 0.667,
 'aggression': 0.554,
 'constitution': 0.443,
 'health': 0.5,
 'isInFaction': '1234234304432',
 'industry': 0.4985,
 'wealth': 0.5828,
 'factionLoyalty': 0.459,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '8880560711424'}

In [5]:
validator = actions.ActionValidator(pop, all_actions)

In [6]:
pd.DataFrame(validator.validate())

,type,label,applies_to,effort,requires_attr,augments_self_properties,comment,leadingdesire
0,patriot_education,action,pop,10,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.01, 'ag...",expand public education programs with a collec...,NaN
1,safety_nets,action,pop,15,{'wealth': 0.1},"{'constitution': 0.05, 'wealth': '-.05'}",increase the general health of the population,NaN
2,individual_education,action,pop,10,{'wealth': 0.1},"{'conformity': -0.05, 'literacy': 0.1}",increase the general health of the population,NaN
3,patriot_propoganda,action,pop,10,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt...",NaN
4,build_infrastructure,action,pop,20,{'wealth': 0.5},"{'wealth': -0.05, 'industrial_infrastructure': 1}",increase industry by investing in local infras...,wealth
5,healthcare_initialtives,action,pop,15,{'wealth': 0.1},"{'health': 0.01, 'wealth': '-.01'}",increase the health and literacy of other popu...,NaN
6,individual_education,action,pop,15,{'wealth': 0.1},"{'wealth': '-.1', 'literacy': 0.01, 'factionLo...","expand privatized education, increase literacy...",NaN


## Creating a job

to create a job you need an agent and and action. This is done in the ajax views for that class (e.g. `app/ajaxviews/pop`) because they are a request from the UI.



In [7]:
def create_job(pop, action, universalTime):
    if type(universalTime) == list:
        universalTime = universalTime[0]
    time_to_complete = int(universalTime["currentTime"]) + int(action["effort"])
    # not actually creating the node here, just demo.
    # uid = create_action_node(c,action,pop)
    popToAction = {
        "node1": pop["objid"],
        "node2": "1234567890",
        "label": "takingAction",
        "name": "takingAction",
        "weight": time_to_complete,
        "actionType": action["type"],
        "status": "pending",
    }
    edges = [popToAction]
    return edges


time = {"currentTime": 10000}
create_job(pop, all_actions[0], time)

[{'node1': '8880560711424',
  'node2': '1234567890',
  'label': 'takingAction',
  'name': 'takingAction',
  'weight': 10010,
  'actionType': 'patriot_education',
  'status': 'pending'}]

In [104]:
actions_query = """
    g.V().haslabel('action').as('action')
                .inE('takingAction').has('status','pending').as('job')
                .outV().as('agent')
                .select('action','job','agent')
"""

c.run_query(actions_query)
pd.DataFrame(c.res)

""


All actions require both an agent, job and action. The properties of the action determine how it affects other things. 


In [65]:
def parse_properties(node):
    n = {}
    for k in node["properties"].keys():
        if len(node["properties"][k]) == 1:
            n[k] = node["properties"][k][0]["value"]
    return n


action = parse_properties(pd.DataFrame(c.res).loc[1].action)
action

{'type': 'individual_education',
 'requires_attr': '{wealth: 0.1}',
 'applies_to': 'pop',
 'effort': 15,
 'augments_self_properties': '{wealth: -.1, literacy: 0.01, factionLoyalty: -.01}',
 'comment': 'expand privatized education, increase literacy and wealth at the cost of faction loyalty',
 'name': 'individual education',
 'username': 'BillmanLocal2',
 'objtype': 'action',
 'objid': '4636333262697'}

In [92]:
agent = parse_properties(pd.DataFrame(c.res).loc[1].agent)
agent

{'isIdle': 'True',
 'name': 'Letquamar Mennu',
 'objid': '2570162977799',
 'conformity': 0.527,
 'literacy': 0.473,
 'aggression': 0.442,
 'constitution': 0.576,
 'health': 0.5,
 'isInFaction': '6048684961165',
 'industry': 0.509,
 'wealth': 0.491,
 'factionLoyalty': 0.609,
 'username': 'BillmanLocal2',
 'objtype': 'pop'}

### Augments self properties
Takes a dictionary of properties and augments them. Adds the property if it does not exist. 


In [93]:
augments_self_properties = yaml.safe_load(action["augments_self_properties"])
augments_self_properties

{'wealth': '-.1', 'literacy': 0.01, 'factionLoyalty': '-.01'}

In [94]:
def resolve_augments_self_properties(agent, action):
    agent = agent.copy()
    self_properties = yaml.safe_load(action["augments_self_properties"])
    for p in self_properties.keys():
        agent[p] = agent[p] + float(self_properties[p])
    return agent


agent2 = resolve_augments_self_properties(agent, action)
results = pd.DataFrame([agent, agent2],).T
print("results of: ", action["type"])
results

results of:  individual_education


,0,1
isIdle,True,True
name,Letquamar Mennu,Letquamar Mennu
objid,2570162977799,2570162977799
conformity,0.527,0.527
literacy,0.473,0.483
aggression,0.442,0.442
constitution,0.576,0.576
health,0.5,0.5
isInFaction,6048684961165,6048684961165
industry,0.509,0.509


In [100]:
def query_patch_properties(agent, action):
    query = f"g.V().has('objid','{agent['objid']}')"
    for n in yaml.safe_load(action["augments_self_properties"]):
        query += f".property('{n}',{agent[n]})"
    return query


query_patch_properties(agent, action)

"g.V().has('objid','2570162977799').property('wealth',0.491).property('literacy',0.473).property('factionLoyalty',0.609)"

Veryify that actions are resolved or not

In [105]:
actions_query = """
    g.V().haslabel('action').as('action')
                .inE('takingAction').has('status','resolved').as('job')
                .outV().as('agent')
                .select('action','job','agent')
"""

c.run_query(actions_query)
pd.DataFrame(c.res)

,action,job,agent
0,"{'id': '539f4d55-7dea-487a-8b62-756b0fe0d313',...","{'id': '6e029d81-83c9-4069-afa6-639aef48bd8f',...","{'id': 'eaa24c0e-8dcf-43d5-af89-ca6bb91d3111',..."
1,"{'id': '42011202-a68d-43d8-841f-87c50f42ac58',...","{'id': '5e9dce26-a02b-4972-a1c3-97d1f7d85f16',...","{'id': '083f7cac-dade-4961-b554-ad3009e704fa',..."
2,"{'id': 'c08bb02d-5739-49f4-a8c1-62c905f0bee5',...","{'id': 'e528b285-0e02-41a4-9226-0b94ffb8433c',...","{'id': '0f6f0e7b-4ce4-4390-a455-5f24a859a62d',..."
3,"{'id': 'aa2e3217-0a2b-47e9-abf7-c963ac28a543',...","{'id': 'e3fa5d2e-ce10-40dc-9367-9e95c3bba555',...","{'id': '90bcb328-de86-42d4-a59d-c182cc3eb995',..."
